<a href="https://colab.research.google.com/github/farahelmashad/ML_Models/blob/main/Decision_Trees/Decision_Tree_from_scratch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from collections import Counter


In [2]:
class Node:
    def __init__(self,feature=None, threshold=None, left=None,right=None, value=None):
        self.feature=feature
        self.threshold=threshold
        self.right=right
        self.left=left
        self.value=value # if it's a leaf node (majority for classification, mean for regression)
    def is_leaf_node(self,node):
      return node.value is not None



In [3]:
class DecisionTree:
    def __init__(self, task='classification', criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, n_features=None):
        self.task=task
        if self.task=="regression":
            self.criterion="mse"
        else:
            self.criterion=criterion
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.min_samples_leaf=min_samples_leaf
        self.n_features= len(X[0]) if n_features is None else min(len(X[0]),n_features) # used for random forests
    def fit(self,X,y):
        self.X=X
        self.y=y
        self.n_features=len(X[0])
        self.root=self.build_tree(self.X,self.y,depth=0)
    def build_tree(self, X, y, depth):
        #check for the base case or stopping criteria
        if ( self.max_depth is not None and depth>=self.max_depth
            or len(y)<= self.min_samples_split
            or len(y)<= self.min_samples_leaf
            or np.unique(y).size == 1):
            if self.task=="classification":
                values_counter=Counter(y)
                majority_class=values_counter.most_common(1)[0][0]
                return Node(value=majority_class)
            elif self.task=="regression":
                return Node(value=np.mean(y))


        best_feature, best_threshold=self.find_best_split(X,y)
        X_left,y_left,X_right,y_right=self.split_data(X,y,best_feature,best_threshold)

        left_subtree=self.build_tree(X_left,y_left, depth+1)
        right_subtree=self.build_tree(X_right,y_right, depth+1)

        return Node(feature=best_feature, threshold=best_threshold,left=left_subtree, right=right_subtree)

    def find_best_split(self, X, y):
        best_gini=float('inf')
        best_ig=-float('inf')
        best_mse=float('inf')
        best_feature=None
        best_threshold=None
        no_features=len(X[0]) # actual number of feeatures we have
        features= np.random.choice(no_features, self.n_features,replace=False)
        for feature in features:
            # unique_values=np.array(sorted(np.unique(X[:,feature])))
            # midpoints=(unique_values[:-1]+unique_values[1:])/2
            thresholds = np.percentile(X[:, feature], np.linspace(0, 100, 10))
            thresholds = np.unique(thresholds)

            for threshold in thresholds:
                X_left,y_left,X_right,y_right=self.split_data(X,y,feature,threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                 continue

                if self.criterion=="gini":
                 weighted_gini=((len(y_left)/len(y))*self.Gini(y_left))+((len(y_right)/len(y))*self.Gini(y_right))
                 if weighted_gini<best_gini:
                    best_gini=weighted_gini
                    best_feature=feature
                    best_threshold=threshold

                elif self.criterion=="entropy":
                    weighted_entropy=((len(y_left)/len(y))*self.entropy(y_left))+((len(y_right)/len(y))*self.entropy(y_right))
                    information_gain=self.entropy(y)-weighted_entropy
                    if information_gain> best_ig:
                        best_ig=information_gain
                        best_feature=feature
                        best_threshold=threshold
                elif self.task=="regression":
                    weighted_mse=(((len(y_left)/len(y))*self.mse(y_left)))+(((len(y_right)/len(y))*self.mse(y_right)))
                    if weighted_mse<best_mse:
                        best_mse=weighted_mse
                        best_feature=feature
                        best_threshold=threshold

        return best_feature, best_threshold

    def split_data(self, X, y, feature, threshold):
     left = X[:, feature] <= threshold
     right = X[:, feature] > threshold
     X_left = X[left]
     y_left = y[left]
     X_right = X[right]
     y_right = y[right]
     return X_left, y_left, X_right, y_right

    def Gini(self,y):
        if len(y)==0:
            return 0.0
        values, counts=np.unique(y, return_counts=True)
        probabilites= counts/counts.sum()
        return 1-np.sum(probabilites**2)
    def entropy(self,y):
        if len(y)==0:
            return 0.0
        values, counts=np.unique(y, return_counts=True)
        probabilities=counts/counts.sum()
        entropies= -probabilities*np.log2(probabilities+1e-9) # aashan el log 0
        return entropies.sum()
    def mse(self, y):
     if len(y) == 0:
        return 0
     mean = np.mean(y)
     return np.mean((y - mean) ** 2)

    def traverse_tree(self,x,node):
      if node.is_leaf_node(node):
        return node.value
      if x[node.feature]<=node.threshold:
        return self.traverse_tree(x,node.left)
      elif x[node.feature]>node.threshold:
        return self.traverse_tree(x,node.right)


    def predict(self, X):
     y = [self.traverse_tree(x, self.root) for x in X]
     return np.array(y)





## Testing the decision tree for classification against sklearn's implementation:


In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf=DecisionTree(n_features=2)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

from sklearn.tree import DecisionTreeClassifier

clf_sklearn = DecisionTreeClassifier(random_state=42)
clf_sklearn.fit(X_train, y_train)
print("Sklearn accuracy:", accuracy_score(y_test, clf_sklearn.predict(X_test)))


Accuracy: 1.00
Sklearn accuracy: 1.0


- we can see that the sklearn accuracy and the implementation accuracies are almost identical =100% , it's high bc the iris dataset is well cleaned, small, and linearly separable


## Testing the decision tree for regression against sklearn's implementation:


In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import fetch_california_housing
import time
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
start_time=time.time()
dtree=DecisionTree(task="regression", criterion="mse",max_depth=10, min_samples_split=10, min_samples_leaf=5)
dtree.fit(X_train,y_train)
y_pred=dtree.predict(X_test)
print("MSE of our model: ", mean_squared_error(y_test, y_pred))
end_time=time.time()
print("Time of our model to train: ", end_time-start_time)

start_time=time.time()
dtreee=DecisionTreeRegressor(max_depth=10, min_samples_split=10, min_samples_leaf=5)
dtreee.fit(X_train,y_train)
y_pred_sk=dtreee.predict(X_test)
end_time=time.time()

print("MSE of sklearn: ", mean_squared_error(y_test, y_pred_sk))
print("Time of sklearn's model to train: ", end_time-start_time)


MSE of our model:  0.4199474349385503
Time of our model to train:  3.7840235233306885
MSE of sklearn:  0.4069562378184981
Time of sklearn's model to train:  0.14514827728271484
